In [1]:
# pip install spotipy 

# Imports

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd
import numpy as np 
import time 

# Credentials

Credentials were stored in another file and I'm going to be importing them.

In [3]:
creds= pd.read_csv('../credentials')
SPOTIPY_CLIENT_ID = creds['Client ID'][0]
SPOTIPY_CLIENT_SECRET = creds['Client Secret'][0]
#Shameless redirect to my dance group lol 
SPOTIPY_REDIRECT_URI= 'https://www.ballethermosoamanecer.com/'
username = ""

# Data Collection 

## Testing 20 Songs in Liked 

In [4]:
client_credentials_manager = SpotifyClientCredentials(client_id= SPOTIPY_CLIENT_ID, 
                                                      client_secret=SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager= client_credentials_manager)
scope = 'user-library-read playlist-read-private playlist-modify-private user-read-recently-played user-top-read'
token = util.prompt_for_user_token(username, scope, 
                                   client_id= SPOTIPY_CLIENT_ID, 
                                   client_secret= SPOTIPY_CLIENT_SECRET, 
                                   redirect_uri= SPOTIPY_REDIRECT_URI)
if token:
    
    sp = spotipy.Spotify(auth=token)
    results = sp.current_user_saved_tracks()
    count = 0
    total_liked_songs = results['total']
    for item in results['items']:
        track = item['track']
        print(track['name'] + ' - ' + track['artists'][0]['name'])
        count += 1
    print(f"\nShowing {count} out of {total_liked_songs} songs")
else:
    print("Can't get token for", username)

Eres Mi Sueño - Septeto Acarey
La Dosis Perfecta - Panteon Rococo
Nunca Es Suficiente - Los Angeles Azules
Las Amo a Las Dos - Cumbias Poblanas
El Esqueleto - Victimas Del Doctor Cerebro
Devuélveme a mi chica - Hombres G
A Través Del Vaso - Banda Los Sebastianes
Cyanide - Metallica
Disco Drum - Black Caviar
Gyal You A Party Animal - Remix - Charly Black
Atrévete - Nicky Jam
Callaita - Bad Bunny
Rip It Up - Little Richard
Butterflies - Samsa
She Felt Like (You) - Hvrbie
Egyptian Luvr (feat. Aminé and Dana Williams) - Rejjie Snow
Asymptotic - Louie Zong
Feels Like Summer - Childish Gambino
Erase - Omar Apollo
Green Light - Lee Foss

Showing 20 out of 986 songs


Clealy my listening habits are... a little diverse haha 

## Collecting Audio Features for Liked Songs

In [5]:
song_name = []
song_uri = []
song_popularity = []
artist_name = []
artist_uri = []
genres_found = []
loops= 0 
track_audio_features = []
while results:
    for item in results['items']:
        track = item['track']
        song_name.append(track['name'])
        song_uri.append(track['uri'])
        song_popularity.append(track['popularity'])
    
        #Some songs have multuple artists so I'll make a list of artists as well as a list of uris for each song
        artist_name.append([artist['name'] for artist in track['artists']])
        artist_uri.append([artist['uri'] for artist in track['artists']])
    
        #Spotify doesn't provide the genre of the song so I have to look at the genre of the artist(s) of the song
        temp = sp.artists([artist['uri'] for artist in track['artists']])['artists']
        temp2 = []
        for temp_artists in temp:
            for genre in temp_artists['genres']:
                temp2.append(genre)
        #We only want to keep unique genres and avoid duplicate genres
        genres_found.append(list(set(temp2)))
    
    #Collecting audio features available for each song 
    #As shown from above, 20 
    track_audio_features.extend(sp.audio_features(song_uri[20*loops :20*(loops+1)]))
    print(f'Audio Features were gathered for {len(track_audio_features)} songs')

    loops += 1 
    results = sp.next(results)
    if (loops % 5) == 0 :
        print(f'{loops} cycles have been completed')
    #Sleep in order to not bombard with too many requests
    time.sleep(.8)

#Bringing it all together 
df_audio_feat = pd.DataFrame(track_audio_features) 
df_tracks = pd.DataFrame({'artist_uri': artist_uri,
                          'song_uri': song_uri,
                          'song_name':song_name,
                          'artists': artist_name,
                          'genres': genres_found,
                          'popularity': song_popularity})

print("\nCreating Dataframe . . .")
collected_data = pd.merge(df_tracks, df_audio_feat, left_on= 'song_uri', right_on= 'uri')
                          

Audio Features were gathered for 20 songs
Audio Features were gathered for 40 songs
Audio Features were gathered for 60 songs
Audio Features were gathered for 80 songs
Audio Features were gathered for 100 songs
5 cycles have been completed
Audio Features were gathered for 120 songs
Audio Features were gathered for 140 songs
Audio Features were gathered for 160 songs
Audio Features were gathered for 180 songs
Audio Features were gathered for 200 songs
10 cycles have been completed
Audio Features were gathered for 220 songs
Audio Features were gathered for 240 songs
Audio Features were gathered for 260 songs
Audio Features were gathered for 280 songs
Audio Features were gathered for 300 songs
15 cycles have been completed
Audio Features were gathered for 320 songs
Audio Features were gathered for 340 songs
Audio Features were gathered for 360 songs
Audio Features were gathered for 380 songs
Audio Features were gathered for 400 songs
20 cycles have been completed
Audio Features were gathe

In [6]:
collected_data.head()

,artist_uri,song_uri,song_name,artists,genres,popularity,danceability,energy,key,loudness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,[spotify:artist:5Vz74ibGHBQaUa2ALDOH0v],spotify:track:5ZA9ItQp7DyG6pPkSsq3Bn,Eres Mi Sueño,[Septeto Acarey],[salsa peruana],62,0.626,0.801,3,-5.144,...,0.2030,0.903,169.956,audio_features,5ZA9ItQp7DyG6pPkSsq3Bn,spotify:track:5ZA9ItQp7DyG6pPkSsq3Bn,https://api.spotify.com/v1/tracks/5ZA9ItQp7DyG...,https://api.spotify.com/v1/audio-analysis/5ZA9...,300773,4
1,[spotify:artist:11mqrDSFRRz8g0Wb3syJj5],spotify:track:7qKtcEDbth6kOM5s4uJzw3,La Dosis Perfecta,[Panteon Rococo],"[latin rock, latin alternative, mexican rock, ...",66,0.676,0.778,0,-5.979,...,0.0316,0.964,155.325,audio_features,7qKtcEDbth6kOM5s4uJzw3,spotify:track:7qKtcEDbth6kOM5s4uJzw3,https://api.spotify.com/v1/tracks/7qKtcEDbth6k...,https://api.spotify.com/v1/audio-analysis/7qKt...,255507,4
2,"[spotify:artist:0ZCO8oVkMj897cKgFH7fRW, spotif...",spotify:track:0HlMshB5JmZjPNbOuOgFHN,Nunca Es Suficiente,"[Los Angeles Azules, Natalia Lafourcade]","[latin rock, latin alternative, gruperas inmor...",74,0.796,0.499,2,-6.166,...,0.0867,0.798,93.990,audio_features,0HlMshB5JmZjPNbOuOgFHN,spotify:track:0HlMshB5JmZjPNbOuOgFHN,https://api.spotify.com/v1/tracks/0HlMshB5JmZj...,https://api.spotify.com/v1/audio-analysis/0HlM...,266016,4
3,[spotify:artist:5yfgm29nhL4CSbKZSPORu4],spotify:track:0jhtqu6qU7UU4qEfQuv634,Las Amo a Las Dos,[Cumbias Poblanas],[cumbia sonidera],59,0.803,0.582,11,-7.171,...,0.3800,0.846,104.766,audio_features,0jhtqu6qU7UU4qEfQuv634,spotify:track:0jhtqu6qU7UU4qEfQuv634,https://api.spotify.com/v1/tracks/0jhtqu6qU7UU...,https://api.spotify.com/v1/audio-analysis/0jht...,289802,4
4,[spotify:artist:6Z112eJxKl1E3nAbYZBr7M],spotify:track:1Z05BEFLTKxdp9xSjUnPGX,El Esqueleto,[Victimas Del Doctor Cerebro],"[latin rock, mexican rock, rock en espanol, la...",57,0.543,0.781,7,-9.249,...,0.5140,0.700,150.395,audio_features,1Z05BEFLTKxdp9xSjUnPGX,spotify:track:1Z05BEFLTKxdp9xSjUnPGX,https://api.spotify.com/v1/tracks/1Z05BEFLTKxd...,https://api.spotify.com/v1/audio-analysis/1Z05...,221640,4


Alright, now there are two options. 

Ideally, I want to create playlists for each genre and place the correct songs into each playlist. Given that some songs can be classified under multiple genres (since we're using the artist's genere as a proxy), it might be difficult to classify songs into genre's that spotify has provided. To start, I will simplify my problem and try classying my music as either a Hispanic genere or not. Since the lyrics are not included, let's see how my model will do using the audio features we collected.


# Data Cleaning

## Outliers and  `null` values

We have some data cleaning to do before I can use my data. Let's check for `null` values and remove columns that are repetitive and provide no additional information. Domain knowledge usually helps identify any outliers or data points that might be off due to human error but for this case, I will assume all values provided by Spotify are normal. 

In [7]:
collected_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 986 entries, 0 to 985
Data columns (total 24 columns):
artist_uri          986 non-null object
song_uri            986 non-null object
song_name           986 non-null object
artists             986 non-null object
genres              986 non-null object
popularity          986 non-null int64
danceability        986 non-null float64
energy              986 non-null float64
key                 986 non-null int64
loudness            986 non-null float64
mode                986 non-null int64
speechiness         986 non-null float64
acousticness        986 non-null float64
instrumentalness    986 non-null float64
liveness            986 non-null float64
valence             986 non-null float64
tempo               986 non-null float64
type                986 non-null object
id                  986 non-null object
uri                 986 non-null object
track_href          986 non-null object
analysis_url        986 non-null object
duration_

## Dropping Unecessary Columns

In [8]:
data= collected_data.drop(columns= ['artist_uri', 'type','id', 'uri', 'track_href', 'analysis_url'])
data[40:52]

,song_uri,song_name,artists,genres,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
40,spotify:track:7uH27oIt4a6cIFCA8ZPcyG,Baila Baila Baila,[Ozuna],"[trap latino, reggaeton, latin, puerto rican pop]",54,0.816,0.754,11,-2.750,0,0.1110,0.03610,0.000000,0.2170,0.533,99.966,158400,4
41,spotify:track:27SdWb2rFzO6GWiYDBTD9j,Cheap Thrills,[Sia],"[australian dance, pop, dance pop, australian ...",76,0.628,0.698,6,-5.608,0,0.1050,0.04720,0.001430,0.0907,0.732,89.976,211667,4
42,spotify:track:3bidbhpOYeV4knp8AIu8Xn,Can't Hold Us - feat. Ray Dalton,[Macklemore & Ryan Lewis],"[dance pop, pop, pop rap]",81,0.641,0.922,2,-4.457,1,0.0786,0.02910,0.000000,0.0862,0.847,146.078,258343,4
43,spotify:track:0FE9t6xYkqWXU2ahLh6D8X,Shape of You,[Ed Sheeran],"[pop, uk pop]",74,0.825,0.652,1,-3.183,0,0.0802,0.58100,0.000000,0.0931,0.931,95.977,233713,4
44,spotify:track:10Yzn7D4l9imoN802Tyg0i,Poolside,[BAYNK],"[new french touch, tropical house, indie elect...",51,0.853,0.484,0,-9.103,1,0.0692,0.63700,0.012500,0.0944,0.586,119.508,212385,4
45,spotify:track:2WlOO5UM3eoEQt7Z2z3PU1,Might Be,[Anderson .Paak],[escape room],54,0.684,0.585,2,-7.601,1,0.0712,0.35700,0.000001,0.7280,0.335,148.507,237166,4
46,spotify:track:3jDdpx9PMlfMBS5tOBHFm9,Return of the Mack,[Mark Morrison],"[british soul, new jack swing]",71,0.715,0.833,2,-5.379,1,0.1080,0.00631,0.000000,0.1640,0.612,95.487,213093,4
47,spotify:track:2x5Frs6ts14U4MSLrQQlXg,Why iii Love The Moon.,[Phony Ppl],"[alternative r&b, neo r&b, indie soul]",63,0.706,0.483,1,-7.929,1,0.0393,0.54700,0.000002,0.1360,0.663,92.002,331827,4
48,spotify:track:5eEQCfq7MXQr6aHLNRUZLs,Keii,[Anuel AA],"[trap latino, reggaeton flow, latin]",80,0.708,0.797,0,-3.095,0,0.0391,0.40000,0.000000,0.0960,0.510,88.015,210000,4
49,spotify:track:7JabxjX6a5GvDDw6zRFscj,Whine Up,"[Nicky Jam, Anuel AA]","[trap latino, tropical, latin, latin hip hop, ...",37,0.817,0.665,9,-6.224,0,0.0758,0.05490,0.000000,0.1030,0.662,104.957,215460,4


## Checking for Duplicates 

In [9]:
data[data.duplicated(subset=['song_uri'])]

,song_uri,song_name,artists,genres,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature


No duplicates found

## Simplifying Generes

Lets see what genres we have:

In [10]:
genre_dummied = data['genres'].str.join(sep='*').str.get_dummies(sep='*')
genre_dummied

,adult standards,african percussion,afro dancehall,afropop,album rock,alternative dance,alternative metal,alternative r&b,alternative rock,argentine hip hop,...,venezuelan rock,veracruz indie,viral pop,vocal jazz,welsh metal,west coast rap,world worship,worship,wrestling,yacht rock
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
982,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
983,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
984,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In theory, I would have 368 different playlists if I made a playlist for each genre. That's excessive and unproductive since some songs pertain to multiple genres and some genres only have one song associated with it. Lets add up our genres and see what's popular in my liked songs 

In [11]:
genre_dummied.sum().sort_values(ascending= False)

latin               325
reggaeton           216
tropical            214
grupera             143
latin pop           132
                   ... 
london rap            1
lo-fi beats           1
lgbtq+ hip hop        1
kids dance party      1
lovers rock           1
Length: 368, dtype: int64

Given that `latin` is the most common genere, I will go back to `data` and check if any `latin` is in the list of generes. I will replace the list with `latin` in order to simplfy my process. There are multiple subgeneres like, cumbia, bachata, merenguem etc. but for the initial phase, I am focusing on latin or not latin.

In [12]:
for i in range(len(genres_found)):
    if {'porro','bachata, merengue','salsa', 'salsa peruana', 'tropical', 'timba'}  & set(genres_found[i]):
        genres_found[i] = ['Bailes del Carribe']
        
    if {'trival','cumbia','cumbia salvadorena','gruperas inmortales' ,'cumbia villera', 'cumbia sonidera', 'tejano', 'guaracha', 'cumbia paraguaya','nu-cumbia', 'grupera'} & set(genres_found[i]):
        genres_found[i] = ['Cumbia']
        
    if {'rap dominicano','perreo','venezuelan hip hop','chilean hardcore','reggaeton', 'dembow', 'trap latino', 'latin hip hop','panamanian pop','venezuelan indie','latin pop', 'latin', 'mexican edm', 'pop romantico'} & set(genres_found[i]):
        genres_found[i] = ['Latin Perreo and Pop']
        
    if {'tamborazo','banda', 'ranchera', 'mariachi', 'duranguense', 'cancion melodica', 'norteno-sax','regional mexican','regional mexican pop'} & set(genres_found[i]):
        genres_found[i] = ['Regional Mexicana']
        
        
    if {'escape room','christian music','shimmer pop','bass trap' ,'emo rap','indie pop','indie soul', 'edm', 'house', 'british soul', 'disco', 'funk','la pop','uk hip hop','social media pop','afropop','reggae fusion','ghanaian pop','soca','hip hop','pop rap', 'pop', 'hip pop', 'rap', 'chicago rap','chicago pop', 'meme rap'} & set(genres_found[i]):
        genres_found[i] = ['Non Hispanic']
        

    if {'mexican rock', 'rock en espanol', 'latin rock', 'mexican rock-and-roll','metalcore','screamo','rock', 'rock-and-roll', 'punk', 'alternative metal', 'soft rock', 'glam rock', 'piano rock'} & set(genres_found[i]):
        genres_found[i] = ['Rock']


data['genres'] = genres_found

In [13]:
data['genres']

0      [Bailes del Carribe]
1                    [Rock]
2                  [Cumbia]
3                  [Cumbia]
4                    [Rock]
               ...         
981    [Bailes del Carribe]
982    [Bailes del Carribe]
983    [Bailes del Carribe]
984    [Bailes del Carribe]
985    [Bailes del Carribe]
Name: genres, Length: 986, dtype: object

In [14]:
genre_dummied = data['genres'].str.join(sep='*').str.get_dummies(sep='*')
genre_dummied

,Bailes del Carribe,Cumbia,Latin Perreo and Pop,Non Hispanic,Regional Mexicana,Rock
0,1,0,0,0,0,0
1,0,0,0,0,0,1
2,0,1,0,0,0,0
3,0,1,0,0,0,0
4,0,0,0,0,0,1
...,...,...,...,...,...,...
981,1,0,0,0,0,0
982,1,0,0,0,0,0
983,1,0,0,0,0,0
984,1,0,0,0,0,0


In [15]:
genre_dummied.sum().sort_values(ascending= False)

Cumbia                  263
Bailes del Carribe      219
Non Hispanic            175
Latin Perreo and Pop    111
Rock                    105
Regional Mexicana        46
dtype: int64

After a lot of EDA, I had to modify my problem since my songs are overwhelmingly Hispanic. I made subdivisions of the different subgenres within Hispanic music. Not accurate but this is how I distinguish my own music tastes and this is how I want my playlists to be organized by (for the most part). It was simplified a bit more but 

In [16]:
#     playlists = sp.user_playlist_create(username, playlist_name,
#                                         description=playlist_description)
# results = sp.user_playlist_add_tracks(username, playlist_id, track_ids)